In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#web scrape wikipedia for list of presidential candidates with party info and total popular vote
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_United_States_presidential_candidates_by_number_of_votes_received"
response = requests.get(url)
response.status_code

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
candidate = []
year = []
party = []
popular_votes = []
winner = []

num_iter = len(soup.select('table:nth-child(3) td'))
for i in range(0,num_iter,5):
    candidate.append(soup.select('table:nth-child(3) td')[i].get_text().split(',')[1].strip())
    year.append(soup.select('table:nth-child(3) td')[i+1].get_text().strip())
    party.append(soup.select('table:nth-child(3) td')[i+2].get_text().strip())
    popular_votes.append(soup.select('table:nth-child(3) td')[i+3].get_text().split('[')[0])
    winner.append(soup.select('table:nth-child(3) td')[i+4].get_text().split('.')[0])
    



In [ ]:
us_presidential = pd.DataFrame({"candidate":candidate, "year":year, "party":party, "popular_vote":popular_votes, "winner":winner})

In [ ]:
us_presidential['year'] = us_presidential['year'].astype('int64')
us_presidential = us_presidential[us_presidential['year'] >= 2006]
us_presidential.sort_values(by=['year'], ascending=False, inplace=True)

In [ ]:
us_presidential['popular_vote'] = [int(x.replace(',','')) for x in us_presidential['popular_vote']]

In [ ]:
us_presidential.reset_index(drop=True,inplace=True)

In [ ]:
def name_winner(x):
    if 'Winner' in x:
        return 1
    else:
        return 0

In [ ]:
us_presidential['winner'] = [name_winner(x) for x in us_presidential['winner']]

In [ ]:
def make_ppv(x):
    yr = us_presidential[us_presidential['year']==us_presidential['year'][x]]
    tpv = yr['popular_vote'].sum()
    ppv = us_presidential['popular_vote'][x]/tpv
    return ppv

In [ ]:
us_presidential['percent_popular_vote'] = list(map(make_ppv,range(len(us_presidential))))
us_presidential.drop('popular_vote', axis=1, inplace=True)
us_presidential

In [ ]:
def clean_names(x):
    if x[0]==x[1]:
        return x[1:]
    else:
        return x
    

In [ ]:
us_presidential['candidate'] = list(map(clean_names, us_presidential['candidate']))

In [ ]:
us_presidential['winner'] = us_presidential['winner'].astype('object')

In [ ]:
us_presidential.dtypes

# gubernatorial elections

In [ ]:
url = "https://en.wikipedia.org/wiki/2020_United_States_gubernatorial_elections"
response = requests.get(url)
response.status_code

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
#mw-content-text > div.mw-parser-output > table:nth-child(30)
t1 = soup.select('table:nth-child(30) tbody tr')
t1[6].select('th a')[0].get_text()

In [ ]:
states = []
candidates = []

for i in range(2,len(t1)):
    candidates.append(t1[i].select('td')[4].get_text().strip())
    states.append(t1[i].select('th a')[0].get_text())


In [ ]:
g_20 = pd.DataFrame({'state':states, 'candidate':candidates})

In [ ]:
drop_list = []
for i in range(len(g_20)):
    c_list = g_20['candidate'].iloc[i].split('%')
    for j in c_list:
        if j:
            r = {'candidate':j, 'state':g_20['state'][i]}
            #print(r)
            g_20 = g_20.append(r, ignore_index=True)
            drop_list.append(i)
g_20.drop(drop_list,axis=0,inplace=True)
g_20.reset_index(drop=True, inplace=True)
                   

In [ ]:
g_20['winner'] = 0
g_20['party'] = 0
g_20['percent_popular_vote'] = 0

for i in range(len(g_20)):
    l = g_20['candidate'][i].split(' ')
    if l[0] == 'Y':
        g_20['winner'].iloc[i] = 1
        l.pop(0)
    g_20['percent_popular_vote'][i] = l[-1]
    l.pop(-1)
    g_20['party'][i] = l[-1][1]
    l.pop(-1)
    g_20['candidate'][i] = ' '.join(l)
    

In [ ]:
g_20['year'] = 2020
g_20 = g_20[['candidate', 'year', 'state', 'party', 'percent_popular_vote', 'winner']]

In [ ]:
g_20

In [ ]:
def get_gubes(year, selector):

    url = f"https://en.wikipedia.org/wiki/{year}_United_States_gubernatorial_elections"
    response = requests.get(url)
    if response.status_code != 200:
        print('response status: ',response.status_code)
    
    soup = BeautifulSoup(response.content, "html.parser")
    t1 = soup.select(f'{selector} tbody tr')

    states = []
    candidates = []

    for i in range(2,len(t1)):
        candidates.append(t1[i].select('td')[4].get_text().strip())
        states.append(t1[i].select('th a')[0].get_text())
    
    df = pd.DataFrame({'state':states, 'candidate':candidates})
    df['year'] = year
    return df

In [ ]:
g_19 = get_gubes(2019,'table:nth-child(14)')
g_19

In [ ]:
import re

def clean_gubes(df):
    drop_list = []
    for i in range(len(df)):
        c_list = df['candidate'].iloc[i]
        c_list = re.sub('\[\w+\]','',c_list)
        c_list = re.sub('\s',' ',c_list)
        c_list = c_list.split('%')
        for j in c_list:
            if j:
                r = {'candidate':j, 'state':df['state'][i], 'year':df['year'][i]}
                df = df.append(r, ignore_index=True)
        drop_list.append(i)
    df.drop(drop_list,axis=0,inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    df['winner'] = 0
    df['party'] = 0
    df['percent_popular_vote'] = 0

    for i in range(len(df)):
        l = df['candidate'][i].split(' ')
        if l[0] == 'Y':
            df['winner'].iloc[i] = 1
            l.pop(0)
        df['percent_popular_vote'][i] = l[-1]
        l.pop(-1)
        df['party'][i] = l[-1][1:-1]
        l.pop(-1)
        df['candidate'][i] = ' '.join(l)
    
    df= df[['candidate', 'year', 'state', 'party', 'percent_popular_vote', 'winner', ]]
    return df

In [ ]:
g_19 = clean_gubes(g_19)
g_19

In [ ]:
def get_gubes2(year, selector):

    url = f"https://en.wikipedia.org/wiki/{year}_United_States_gubernatorial_elections"
    response = requests.get(url)
    if response.status_code != 200:
        print('response status: ',response.status_code)
    
    soup = BeautifulSoup(response.content, "html.parser")
    t1 = soup.select(f'{selector} tbody tr')

    states = []
    candidates = []

    for i in range(1,len(t1)):
        candidates.append(t1[i].select('td')[5].get_text().strip())
        states.append(t1[i].select('td a')[0].get_text())
    
    df = pd.DataFrame({'state':states, 'candidate':candidates})
    df['year'] = year
    return df

In [ ]:
g_18 = get_gubes2(2018,'table:nth-child(24)')
g_18

In [ ]:
g_18 = clean_gubes(g_18)
g_18

In [ ]:
#2017 does not have info on percent popular vote

In [ ]:
g_17 = get_gubes2(2017,'table:nth-child(11)')
g_17

In [ ]:
g_17['candidate'][0]

In [ ]:
def get_gubes3(year, selector):

    url = f"https://en.wikipedia.org/wiki/{year}_United_States_gubernatorial_elections"
    response = requests.get(url)
    if response.status_code != 200:
        print('response status: ',response.status_code)
    
    soup = BeautifulSoup(response.content, "html.parser")
    t1 = soup.select(f'{selector} tbody tr')

    states = []
    candidates = []

    for i in range(2,len(t1)):
        candidates.append(t1[i].select('td')[5].get_text().strip())
        states.append(t1[i].select('th a')[0].get_text())
    
    df = pd.DataFrame({'state':states, 'candidate':candidates})
    df['year'] = year
    return df

In [ ]:
g_16 = get_gubes3(2016,'table:nth-child(13)')
g_16 = clean_gubes(g_16)
g_16

In [ ]:
g_15 = get_gubes(2015,'table:nth-child(9)')
g_15 = clean_gubes(g_15)
g_15

In [ ]:
g_14 = get_gubes(2014,'table:nth-child(18)')
g_14 = clean_gubes(g_14)
g_14

In [ ]:
def get_gubes4(year, selector):

    url = f"https://en.wikipedia.org/wiki/{year}_United_States_gubernatorial_elections"
    response = requests.get(url)
    if response.status_code != 200:
        print('response status: ',response.status_code)
    
    soup = BeautifulSoup(response.content, "html.parser")
    t1 = soup.select(f'{selector} tbody')

    states = []
    candidates = []

    for i in range(0,len(t1)-1):
        states.append(t1[0].select('th a')[i].get_text())
        c = t1[0].select('td ul')[i]
        c2 = [c.select('li')[x].get_text() for x in range(len(c.select('li')))]
        c3 = ''.join(c2)
        candidates.append(c3)
    
    df = pd.DataFrame({'state':states, 'candidate':candidates})
    df['year'] = year
    return df

In [ ]:
g_13 = get_gubes4(2013,'table.wikitable')
g_13 = clean_gubes(g_13)
g_13

In [ ]:
url = f"https://en.wikipedia.org/wiki/2012_United_States_gubernatorial_elections"
response = requests.get(url)
if response.status_code != 200:
    print('response status: ',response.status_code)
    
soup = BeautifulSoup(response.content, "html.parser")
t1 = soup.select('table.wikitable tbody')[1]
t2 = t1.select('tr')

states = []
candidates = []

for i in range(1,len(t2)):
    states.append(t2[i].select('th a')[0].get_text())
    c = t2[i].select('td ul li')
    c2 = [c[x].get_text() for x in range(len(c))]
    c3 = ''.join(c2)
    candidates.append(c3)

g_12 = pd.DataFrame({'state':states, 'candidate':candidates})
g_12['year'] = 2012
g_12 = clean_gubes(g_12)
g_12

In [ ]:
url = f"https://en.wikipedia.org/wiki/2011_United_States_gubernatorial_elections"
response = requests.get(url)
if response.status_code != 200:
    print('response status: ',response.status_code)
    
soup = BeautifulSoup(response.content, "html.parser")
t1 = soup.select('table.wikitable tbody')[1]
t2 = t1.select('tr')

states = []
candidates = []

for i in range(1,len(t2)):
    states.append(t2[i].select('th a')[0].get_text())
    c = t2[i].select('td ul li')
    c2 = [c[x].get_text() for x in range(len(c))]
    c3 = ''.join(c2)
    candidates.append(c3)

g_11 = pd.DataFrame({'state':states, 'candidate':candidates})
g_11['year'] = 2011
g_11 = clean_gubes(g_11)
g_11

In [ ]:
url = f"https://en.wikipedia.org/wiki/2010_United_States_gubernatorial_elections"
response = requests.get(url)
if response.status_code != 200:
    print('response status: ',response.status_code)
    
soup = BeautifulSoup(response.content, "html.parser")
t1 = soup.select('table.wikitable tbody')[1]
t2 = t1.select('tr')

states = []
candidates = []

for i in range(1,len(t2)):
    states.append(t2[i].select('th a')[0].get_text())
    c = t2[i].select('td ul li')
    c2 = [c[x].get_text() for x in range(len(c))]
    c3 = ''.join(c2)
    candidates.append(c3)

g_10 = pd.DataFrame({'state':states, 'candidate':candidates})
g_10['year'] = 2010
g_10 = clean_gubes(g_10)
g_10

In [ ]:
url = f"https://en.wikipedia.org/wiki/2009_United_States_gubernatorial_elections"
response = requests.get(url)
if response.status_code != 200:
    print('response status: ',response.status_code)
    
soup = BeautifulSoup(response.content, "html.parser")
t1 = soup.select('table.wikitable tbody')[1]
t2 = t1.select('tr')

states = []
candidates = []

for i in range(1,len(t2)):
    states.append(t2[i].select('th a')[0].get_text())
    c = t2[i].select('td ul li')
    c2 = [c[x].get_text() for x in range(len(c))]
    c3 = ''.join(c2)
    candidates.append(c3)

g_09 = pd.DataFrame({'state':states, 'candidate':candidates})
g_09['year'] = 2009
g_09 = clean_gubes(g_09)
g_09

In [ ]:
url = f"https://en.wikipedia.org/wiki/2008_United_States_gubernatorial_elections"
response = requests.get(url)
if response.status_code != 200:
    print('response status: ',response.status_code)
    
soup = BeautifulSoup(response.content, "html.parser")
t1 = soup.select('table.wikitable tbody')[0]
t2 = t1.select('tr')

states = []
candidates = []

for i in range(1,len(t2)):
    states.append(t2[i].select('th a')[0].get_text())
    c = t2[i].select('td ul li')
    c2 = [c[x].get_text() for x in range(len(c))]
    c3 = ''.join(c2)
    candidates.append(c3)

g_08 = pd.DataFrame({'state':states, 'candidate':candidates})
g_08['year'] = 2008
g_08 = clean_gubes(g_08)
g_08

In [ ]:
url = f"https://en.wikipedia.org/wiki/2007_United_States_gubernatorial_elections"
response = requests.get(url)
if response.status_code != 200:
    print('response status: ',response.status_code)
    
soup = BeautifulSoup(response.content, "html.parser")
t1 = soup.select('table.wikitable tbody')[0]
t2 = t1.select('tr')

states = []
candidates = []

for i in range(1,len(t2)):
    states.append(t2[i].select('th a')[0].get_text())
    c = t2[i].select('td ul li')
    c2 = [c[x].get_text() for x in range(len(c))]
    c3 = ''.join(c2)
    candidates.append(c3)

g_07 = pd.DataFrame({'state':states, 'candidate':candidates})
g_07['year'] = 2007
g_07 = clean_gubes(g_07)
g_07

In [ ]:
url = f"https://en.wikipedia.org/wiki/2006_United_States_gubernatorial_elections"
response = requests.get(url)
if response.status_code != 200:
    print('response status: ',response.status_code)
    
soup = BeautifulSoup(response.content, "html.parser")
t1 = soup.select('table.wikitable tbody')[1]
t2 = t1.select('tr')

states = []
candidates = []

for i in range(1,len(t2)):
    states.append(t2[i].select('th a')[0].get_text())
    c = t2[i].select('td ul li')
    c2 = [c[x].get_text() for x in range(len(c))]
    c3 = ''.join(c2)
    candidates.append(c3)

g_06 = pd.DataFrame({'state':states, 'candidate':candidates})
g_06['year'] = 2006
g_06 = clean_gubes(g_06)
g_06

In [ ]:
gub = pd.concat([g_20,g_19,g_18,g_16,g_15,g_14,g_13,g_12,g_11,g_10,g_09,g_08,g_07,g_06], axis=0)
gub = gub.reset_index(drop=True)
gub

In [ ]:
def clean_names2(x):
    if len(re.findall('\(', x)) != 0:
        a = x.split('(')[0]   
        return a.strip() 
    else:
        return x

In [ ]:
gub['candidate'] = list(map(clean_names2,gub['candidate']))

In [ ]:
gub.drop([65], axis=0, inplace=True)

In [ ]:
gub = gub.reset_index(drop=True)
gub.head()

In [ ]:
us_presidential.to_csv('pres.csv', index=False)

In [ ]:
gub.to_csv('gub.csv', index=False)